In [36]:
import numpy as np
import matplotlib.pyplot as plt
from stablab import (semicirc, winding_number, Evans_plot, emcset,
                       Evans_compute, Struct, reflect_image)
from stablab.wave_profile import profile_flux, soln
from stablab.root_finding import moments_roots
from stablab.evans import LdimRdim

from MHD2 import profile_ode, profile_jacobian
from MHD2 import A_evans, A_lop, A_jump, Ak

import pickle

np.set_printoptions(suppress=True)

# Initialize Parameters

# Parameters
def setParams(h1,vp):
    cnu = 1
    mu = .1
    Gamma = 2/3
    p = Struct({'h1': h1,
            'Gamma': Gamma, 
            'v_plus': vp,
            'mu': .1,
            'eta': -2/3*mu,
            'cnu': cnu,
            'R': cnu*Gamma,
            'alpha':1,
            'kappa': .1 })
    p.update({
            'v_star': p['Gamma']/(p['Gamma']+2) # 3.14
            })
    p.update({
            'e_plus': p['v_plus']*(p['Gamma']+2-p['Gamma']*p['v_plus'])/(2*p['Gamma']*(p['Gamma']+1)), # 3.15
            'e_minus': (p['Gamma']+2)*(p['v_plus']-p['v_star'])/(2*p['Gamma']*(p['Gamma']+1)), # 3.16
            'v_minus': 1,
            'nu': p['kappa']/p['cnu'] # see below 2.25
            })
    p.update({
            'UR': [1/p['v_plus'],p['v_plus'],0,p['h1'],0,p['e_plus']/p['cnu']],
            'UL': [1/p['v_minus'],p['v_minus'],0,p['h1'],0,p['e_minus']/p['cnu']]
            })
    return p

def initialize_sol(p):
    # Initialising sol, the Struct with solution values
    sol = Struct({
        'n': 2, # this is the dimension of the profile ode
        # we divide the domain in half to deal with the
        # non-uniqueness caused by translational invariance
        # sol.side = 1 means we are solving the profile on the interval [0,X]
        'side': 1,
        'F': profile_ode, # F is the profile ode
        'Flinear': profile_jacobian, # J is the profile ode Jacobian
        'UL': np.array([p['v_minus'],p['e_minus']]), # These are the endstates of the profile and its derivative at x = -infty
        'UR': np.array([p['v_plus'],p['e_plus']]), # These are the endstates of the profile and its derivative at x = +infty
        'tol': 1e-7,
        'xi': 0.1,
        'system' : 'parallel',
        'mat_type' : 'mbfv',
        'tol':1e-5
        })
    sol.update({
        'phase': 0.5*(sol['UL']+sol['UR']), # this is the phase condition for the profile at x = 0
        'order': [1], # this indicates to which component the phase conditions is applied
        'stats': 'on', # this prints data and plots the profile as it is solved
        'bvp_options': {'Tol': 1e-5, 'Nmax': 200},
        'L_max': 10000,
        'R_max': 10000
        })
    return sol

# Solve Profile
def solve_profile(p,sol):
    sol['stats'] = 'off'
    p,s = profile_flux(p,sol)
    return p,s

# Plot the profile
def plot_profile(p,s):
    x = np.linspace(s['L'],s['R'],10)
    y = soln(x,s)

    plt.figure("Profile")
    plt.plot(x,y)
    plt.show()
    
def set_up(h1,vp):
    p = setParams(h1,vp)
    sol = initialize_sol(p)
    p,s = solve_profile(p,sol)
    
    # plot_profile(p,s)
    return p,s

# Evaluate Evans Function
def evaluate_Evans(p,s,xi):
    R,tol = .01,.01
    ksteps = 2**6;
    lamsteps = 2**8;

    # Set Fourier variable
    s['xi'] = xi


    # Set up Stablab structures
    # s, e, m, c = emcset(s,'front',[5,6],'reg_reg_polar',A)
    Ldim,Rdim = LdimRdim(A_evans, s, p)
    s, e, m, c = emcset(s,'front',[Ldim,Rdim],'default',A_evans)

    # refine the Evans function computation to achieve set relative error
    c['refine'] = 'on';
    c['ksteps'] = ksteps
    c['lambda_steps'] = lamsteps
    c['tol'] = tol
    c['check'] = 'off'

    # display a waitbar
    c['stats'] = 'off'

    # Set up the preimage
    circpnts, imagpnts, innerpnts = 20, 20, 5
    spread = 4
    zerodist = 0#10**(-4)
    
    preimage = semicirc(circpnts, imagpnts, c['ksteps'], R, spread, zerodist,c['lambda_steps'])

    # Compute the Evans function
    out, domain = Evans_compute(preimage,c,s,p,m,e)

    # Normalize and plot the Evans function
    out = out/out[0]
    w = reflect_image(out)
    prew = reflect_image(domain)
    
    # Evans_plot(w,titlestring='Evans Function')
    # plt.show()

    wnd = winding_number(w)
    rts = moments_roots(prew,w).real

#     print('Evans Function Winding Number: {:f}'.format(wnd))
    
    output = {'w':w,'wnd':wnd,'prew':prew,'xi':xi,'R':R,
              'tol':tol,'p':dict(p),'rts':rts}
    return output



# Evaluate Lopatinski Determinant
def evaluate_Lopatinski(p,s):
    R,tol = .1,.1
    ksteps = 2**8
    
    # Set Fourier variable
    s['xi'] = 1

    # Set up Stablab structures
    Ldim,Rdim = LdimRdim(A_lop, s, p)
    s, e, m, c = emcset(s,'lopatinski',[Ldim,Rdim],'default',A_lop)

    # set the lopatinski jump condition
    e['jump'] = A_jump


    s['R'] = 1
    s['L'] = -1

    # refine the Evans function computation to achieve set relative error
    c['refine'] = 'on';
    c['ksteps'] = ksteps
    c['tol'] = tol
    c['check'] = 'off'
    
    # display a waitbar
    c['stats'] = 'off'

    # Set up the preimage
    circpnts, imagpnts, innerpnts = 20, 80, 50
    shift = 1e-5
    spread = 4
    zerodist = 0
    preimage = semicirc(circpnts, imagpnts, c['ksteps'], R, spread, zerodist)+shift

    # Compute the Evans function
    out, domain = Evans_compute(preimage,c,s,p,m,e)

    # Normalize and plot the Evans function
    out = out/out[0]
    w = reflect_image(out)
    prew = reflect_image(domain)
    # Evans_plot(w,titlestring='Lopatinski Determinant')

    wnd = winding_number(w)
    rts = moments_roots(prew,w).real

#     print('Lopatinski Winding Number: {:f}'.format(wnd))

    output = {'w':w,'wnd':wnd,'prew':prew,'xi':1,'R':R,
              'tol':tol,'p':dict(p),'rts':rts}
    return output


    
# # contour radius
# R = 0.01;

# # contour tol
# tol = 0.01;





In [46]:
h1_vals = [0,1]
vp_vals = [.5,1.01]
data = {}

for h1 in h1_vals:
    for vp in vp_vals:
        # key for data storage
        key = str(h1)+' '+str(vp)
        data[key] = {}
        print(key)

        # set up profile and params
        p,s = set_up(h1,vp)

        # lopatinski
        print('\tlopatinki')
        try:
            res = evaluate_Lopatinski(p,s)
        except:
            res = None
        data[key]['lop'] = res

        # evans
        print('\tevans')
        data[key]['evans'] = {}
        for xi in [.1,.2]:
            print('\t\t{}'.format(xi))
            try:
                res = evaluate_Evans(p,s,xi)
            except:
                res = None
            data[key]['evans'][xi] = res


0 0.5
	lopatinki
	evans
		0.1
		0.2
0 1.01
Eigenvalues at negative infinity: 
[-0.07682366 13.01682366]
Eigenvalues at positive infinity: 
[ 0.07626033 12.98314561]


ValueError: profile_flux.m does not solve undercompressive profiles

In [47]:
data

{'0 0.5': {'lop': {'w': array([1.        +0.j        , 0.99948022+0.00989753j,
          0.99792815+0.01967617j, 0.99536548+0.02921879j,
          0.99182808+0.03841181j, 0.98736546+0.04714693j,
          0.98204012+0.05532283j, 0.97592663+0.0628468j ,
          0.96911052+0.06963635j, 0.96168696+0.07562061j,
          0.95375913+0.08074168j, 0.94543651+0.0849557j ,
          0.93683285+0.08823375j, 0.92806399+0.09056241j,
          0.91924561+0.09194403j, 0.91049087+0.09239656j,
          0.90190801+0.09195306j, 0.89359817+0.09066074j,
          0.88565318+0.08857965j, 0.87815383+0.08578097j,
          0.87996729+0.08148282j, 0.88162785+0.07735706j,
          0.88314663+0.07339782j, 0.88453408+0.06959949j,
          0.88580001+0.06595668j, 0.8869536 +0.06246422j,
          0.88800348+0.05911711j, 0.88895771+0.05591056j,
          0.88982381+0.05283996j, 0.89060885+0.04990082j,
          0.89131938+0.04708885j, 0.89196153+0.04439988j,
          0.892541  +0.04182988j, 0.8930631 +0.0393

In [38]:
pickle.dump(data, open("output/test1.pkl","wb"))

In [39]:
load = pickle.load(open("output/test1.pkl","rb"))

In [42]:
load['0 0.3']['lop']['p']

{'h1': 0,
 'Gamma': 0.6666666666666666,
 'v_plus': 0.3,
 'mu': 0.1,
 'eta': -0.06666666666666667,
 'cnu': 1,
 'R': 0.6666666666666666,
 'alpha': 1,
 'kappa': 0.1,
 'v_star': 0.25,
 'e_plus': 0.333,
 'e_minus': 0.06,
 'v_minus': 1,
 'nu': 0.1,
 'UR': [3.3333333333333335, 0.3, 0, 0, 0, 0.333],
 'UL': [1.0, 1, 0, 0, 0, 0.06]}

In [32]:
type(p)

stablab.Struct.Struct

In [34]:
p1 = dict(p)

In [35]:
type(p1)

dict

In [43]:
from importlib import import_module

In [45]:
f = import_module("MHD2")